#### Thanks to Daniel Villaveces' article on *Medium* for helping me to scrape the data from Citi Bike data. 
#### It saved me a lot of time that otherwise would have been spent downloading multiple zip files for each month.
#### Daniel's article can be found [here](https://medium.com/@dvillaveces/analyzing-citi-bike-data-2202f9da97d7)

In [ ]:
# scrape the citi bike index and extract the file names
# the data begins in 2013, but I only want data from 2020

In [2]:
# the files also have a second file named with 'JC' for every month from 2015 - 2020 that seems to contain addtional/
# trip data, so I will have to do this twice: first to get the 'non JC files' and then get the'JC' files

import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://s3.amazonaws.com/tripdata/'

# load all url content into soup
r = requests.get(url)
soup = BeautifulSoup(r.text, 'xml')

# extract file names from soup
# data from 2020
files = soup.find_all('Key')
extract_files = []
for i in range(80, 92):
    extract_files.append(files[i].get_text())
    
extract_files

['202001-citibike-tripdata.csv.zip',
 '202002-citibike-tripdata.csv.zip',
 '202003-citibike-tripdata.csv.zip',
 '202004-citibike-tripdata.csv.zip',
 '202005-citibike-tripdata.csv.zip',
 '202006-citibike-tripdata.csv.zip',
 '202007-citibike-tripdata.csv.zip',
 '202008-citibike-tripdata.csv.zip',
 '202009-citibike-tripdata.csv.zip',
 '202010-citibike-tripdata.csv.zip',
 '202011-citibike-tripdata.csv.zip',
 '202012-citibike-tripdata.csv.zip']

In [4]:
# extract the JC files, 2020
files = soup.find_all('Key')
jc_extract_files = []
for i in range(145, len(files)-2):
    jc_extract_files.append(files[i].get_text())
    
jc_extract_files

['JC-202001-citibike-tripdata.csv.zip',
 'JC-202002-citibike-tripdata.csv.zip',
 'JC-202003-citibike-tripdata.csv.zip',
 'JC-202004-citibike-tripdata.csv.zip',
 'JC-202005-citibike-tripdata.csv.zip',
 'JC-202006-citibike-tripdata.csv.zip',
 'JC-202007-citibike-tripdata.csv.zip',
 'JC-202008-citibike-tripdata.csv.zip',
 'JC-202009-citibike-tripdata.csv.zip',
 'JC-202010-citibike-tripdata.csv.zip',
 'JC-202011-citibike-tripdata.csv.zip',
 'JC-202012-citibike-tripdata.csv.zip']

In [5]:
# download and unzip the files using a for loop

import os
import zipfile

In [8]:
# download & unzip non-JC files
# move data up a directory so it's not added to github

for file in extract_files:
    file_url = url + file
    
    #download files
    with open(file, "wb") as f:
            response = requests.get(file_url)
            f.write(response.content)
    
    # unzip data files
    with zipfile.ZipFile(file, "r") as zip_ref:
        zip_ref.extractall("../tripdata")        
    
    # remove zipped file after unziping
    os.remove(file)

In [9]:
# download & unzip JC files

for file in jc_extract_files:
    file_url = url + file
    
    #download files
    with open(file, "wb") as f:
            response = requests.get(file_url)
            f.write(response.content)
    
    # unzip data files
    with zipfile.ZipFile(file, "r") as zip_ref:
        zip_ref.extractall("../tripdata")        
    
    # remove zipped file after unziping
    os.remove(file)

In [1]:
# rename files to be more manageable
# files have long names such as '202001-citibike-tripdata.csv' or 'JC-202001-citibike-tripdata'
# just want the year and month and also include the 'jc' until I can figure out the difference

import numpy as np
import pandas as pd

In [11]:
directory = '../tripdata/'

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.csv'):
        new_filename = filename.replace(' ','').lower().split('ci', 1)[0].strip('-').replace('-','_')
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename + '.csv'))

In [6]:
#load into DataFrames into dfs dictionary
directory = '../tripdata/'
dfs = {}

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.csv'):
        dfs[filename.split('.')[0]] = pd.read_csv(os.path.join(directory, filename)) 

In [13]:
dfs['202001']

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240591,1587,2020-01-31 23:59:26.8820,2020-02-01 00:25:54.4890,3244,University Pl & E 8 St,40.731437,-73.994903,3092,Berry St & N 8 St,40.719009,-73.958525,40662,Subscriber,1990,1
1240592,222,2020-01-31 23:59:32.6410,2020-02-01 00:03:15.4720,383,Greenwich Ave & Charles St,40.735238,-74.000271,383,Greenwich Ave & Charles St,40.735238,-74.000271,28722,Subscriber,1983,1
1240593,163,2020-01-31 23:59:39.1780,2020-02-01 00:02:23.0400,150,E 2 St & Avenue C,40.720874,-73.980858,411,E 6 St & Avenue D,40.722281,-73.976687,32530,Subscriber,1958,2
1240594,327,2020-01-31 23:59:49.2310,2020-02-01 00:05:16.3790,483,E 12 St & 3 Ave,40.732233,-73.988900,3718,E 11 St & Avenue B,40.727464,-73.979504,15314,Customer,1994,1


In [14]:
type(dfs['202001'])

pandas.core.frame.DataFrame

In [19]:
# merge dfs to create a larger one for each year (should have used a loop?)

twenty = [dfs['202001'],dfs['202002'],dfs['202003'],dfs['202004'],dfs['202005'],dfs['202006'],dfs['202007'],dfs['202008'],dfs['202009'],dfs['202010'],dfs['202011'],dfs['202012'],dfs['jc_202001'],dfs['jc_202002'],dfs['jc_202003'],dfs['jc_202004'],dfs['jc_202005'],dfs['jc_202006'],dfs['jc_202007'],dfs['jc_202008'],dfs['jc_202009'],dfs['jc_202010'],dfs['jc_202011'],dfs['jc_202012']]

twenty_df = pd.concat(twenty)

twenty_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,1


In [16]:
twenty_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11689,1750,2020-12-31 23:07:44.0030,2020-12-31 23:36:54.4710,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,-74.032108,40440,Customer,1969,0
11690,1519,2020-12-31 23:18:00.2630,2020-12-31 23:43:19.8590,3184,Paulus Hook,40.714145,-74.033552,3195,Sip Ave,40.730897,-74.063913,46340,Customer,1969,0
11691,1761,2020-12-31 23:31:09.4620,2021-01-01 00:00:31.3290,3195,Sip Ave,40.730897,-74.063913,3270,Jersey & 6th St,40.725289,-74.045572,40907,Customer,1998,1
11692,637,2020-12-31 23:35:45.4640,2020-12-31 23:46:22.7300,3681,Grand St,40.715178,-74.037683,3199,Newport Pkwy,40.728745,-74.032108,42250,Customer,1969,0


In [20]:
# select only needed columns
# I would like to keep all the columns, but this makes the dataset smaller and more managable.

twenty_df = twenty_df[['tripduration','starttime','start station id', 'start station name','start station latitude','start station longitude','usertype','bikeid','birth year','gender']]
twenty_df.head()

,tripduration,starttime,start station id,start station name,start station latitude,start station longitude,usertype,bikeid,birth year,gender
0,789,2020-01-01 00:00:55.3900,504,1 Ave & E 16 St,40.732219,-73.981656,Subscriber,30326,1992,1
1,1541,2020-01-01 00:01:08.1020,3423,West Drive & Prospect Park West,40.661063,-73.979453,Customer,17105,1969,1
2,1464,2020-01-01 00:01:42.1400,3687,E 33 St & 1 Ave,40.743227,-73.974498,Subscriber,40177,1963,1
3,592,2020-01-01 00:01:45.5610,346,Bank St & Hudson St,40.736529,-74.006180,Subscriber,27690,1980,1
4,702,2020-01-01 00:01:45.7880,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,Subscriber,32583,1982,1


In [21]:
# rename columns

twenty_df.rename(columns = {'tripduration':'duration(s)', 'starttime':'start_time', 'start station id': 'start_id',
                             'start station name': 'start_name', 'start station latitude': 'start_lat', 'start station longitude': 'start_lng',
                             'bikeid': 'bike_id', 'birth year': 'birth_year'}, inplace = True)
twenty_df.head()

,duration(s),start_time,start_id,start_name,start_lat,start_lng,usertype,bike_id,birth_year,gender
0,789,2020-01-01 00:00:55.3900,504,1 Ave & E 16 St,40.732219,-73.981656,Subscriber,30326,1992,1
1,1541,2020-01-01 00:01:08.1020,3423,West Drive & Prospect Park West,40.661063,-73.979453,Customer,17105,1969,1
2,1464,2020-01-01 00:01:42.1400,3687,E 33 St & 1 Ave,40.743227,-73.974498,Subscriber,40177,1963,1
3,592,2020-01-01 00:01:45.5610,346,Bank St & Hudson St,40.736529,-74.006180,Subscriber,27690,1980,1
4,702,2020-01-01 00:01:45.7880,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,Subscriber,32583,1982,1


In [22]:
# rearrange columns
twenty_df = twenty_df[['bike_id','usertype','birth_year','gender', 'start_lat', 'start_lng', 'start_id','start_name', 'duration(s)','start_time']]
twenty_df.head()

,bike_id,usertype,birth_year,gender,start_lat,start_lng,start_id,start_name,duration(s),start_time
0,30326,Subscriber,1992,1,40.732219,-73.981656,504,1 Ave & E 16 St,789,2020-01-01 00:00:55.3900
1,17105,Customer,1969,1,40.661063,-73.979453,3423,West Drive & Prospect Park West,1541,2020-01-01 00:01:08.1020
2,40177,Subscriber,1963,1,40.743227,-73.974498,3687,E 33 St & 1 Ave,1464,2020-01-01 00:01:42.1400
3,27690,Subscriber,1980,1,40.736529,-74.006180,346,Bank St & Hudson St,592,2020-01-01 00:01:45.5610
4,32583,Subscriber,1982,1,40.694546,-73.958014,372,Franklin Ave & Myrtle Ave,702,2020-01-01 00:01:45.7880


In [24]:
# split date from time in 'start_time'
twenty_df[['Date','Time']] = twenty_df.start_time.str.split(expand=True) 

In [25]:
twenty_df.head()

,bike_id,usertype,birth_year,gender,start_lat,start_lng,start_id,start_name,duration(s),start_time,Date,Time
0,30326,Subscriber,1992,1,40.732219,-73.981656,504,1 Ave & E 16 St,789,2020-01-01 00:00:55.3900,2020-01-01,00:00:55.3900
1,17105,Customer,1969,1,40.661063,-73.979453,3423,West Drive & Prospect Park West,1541,2020-01-01 00:01:08.1020,2020-01-01,00:01:08.1020
2,40177,Subscriber,1963,1,40.743227,-73.974498,3687,E 33 St & 1 Ave,1464,2020-01-01 00:01:42.1400,2020-01-01,00:01:42.1400
3,27690,Subscriber,1980,1,40.736529,-74.006180,346,Bank St & Hudson St,592,2020-01-01 00:01:45.5610,2020-01-01,00:01:45.5610
4,32583,Subscriber,1982,1,40.694546,-73.958014,372,Franklin Ave & Myrtle Ave,702,2020-01-01 00:01:45.7880,2020-01-01,00:01:45.7880


In [27]:
# drop columns because I only need the date
twenty_df.drop(twenty_df.columns[[9, 11]], axis = 1, inplace = True) 
twenty_df.head()

,bike_id,usertype,birth_year,gender,start_lat,start_lng,start_id,start_name,duration(s),Date
0,30326,Subscriber,1992,1,40.732219,-73.981656,504,1 Ave & E 16 St,789,2020-01-01
1,17105,Customer,1969,1,40.661063,-73.979453,3423,West Drive & Prospect Park West,1541,2020-01-01
2,40177,Subscriber,1963,1,40.743227,-73.974498,3687,E 33 St & 1 Ave,1464,2020-01-01
3,27690,Subscriber,1980,1,40.736529,-74.006180,346,Bank St & Hudson St,592,2020-01-01
4,32583,Subscriber,1982,1,40.694546,-73.958014,372,Franklin Ave & Myrtle Ave,702,2020-01-01


In [19]:
# check for nulls

twenty_df.count()

duration(s)    19843659
start_time     19843659
stop_time      19843659
start_id       19843659
start_name     19843659
start_lat      19843659
start_lng      19843659
bike_id        19843659
birth_year     19843659
dtype: int64

In [17]:
# export as csv for use in Tableau

twenty_df.to_csv("../clean_tripdata/2020.csv", index=True, header=True)